In [ ]:
# Dive into Plasmapy

### Contents

1. [`plasmapy.formulary` Unit Agnosticism](#plasmapy.formulary-Unit-Agnoticism)
2. [`@validate_quantities` - The Units Enforcer](#@validate_quantites---The-Units-Enforcer)
3. [`@particle_input`](#@particle_input)

## `plasmapy.formulary` Unit Agnoticism

`astropy.unts` (along with some magic) allowed us to develop a formulary that is agnostic to input units, and will always return values in SI units.

In [ ]:
import astropy.units as u

from plasmapy.formulary import Debye_length

Let's start with SI units

In [ ]:
T_e = 115000 * u.K  # about 10 eV
n_e = 2e18 * u.m**-3

Debye_length(T_e=T_e, n_e=n_e)

Let's do the same calculation, but with non-SI units

In [ ]:
T_e_in_eV = T_e.to(u.eV, equivalencies=u.temperature_energy())
n_e_in_lsec3 = n_e.to(u.lsec**-3)

(T_e_in_eV, n_e_in_lsec3)

In [ ]:
Debye_length(T_e=T_e_in_eV, n_e=n_e_in_lsec3)

## `@validate_quantites` - The Units Enforcer

The reasone we have unit agnosticism is because we have develped an units enforce.  This enforcer, `@validate_quantites`, knows how to recognize input unit types, interpret what the function is expecting, and do the proper conversion or enforcement.

Let's start with a function that calculates the kinetic energy of a mass.

In [ ]:
def kinetic_energy_lite(mass, velocity):
    return 0.5 * mass * velocity**2

kinetic_energy_lite(mass=1233, velocity=39.34)

But we have not unit enforement...

In [ ]:
from plasmapy.utils.decorators import validate_quantities

In [ ]:
kinetic_energy = validate_quantities(func=kinetic_energy_lite, mass=u.kg, velocity=u.m/u.s)

kinetic_energy(mass=1233, velocity=39.34)

Now we are getting warnings that input arguments are not given with units, and the output units are a total mess.

In [ ]:
kinetic_energy(mass=1233 * u.kg, velocity=39.34 * u.m/u.s)

The output units are still a mess, so we can do better...

In [ ]:
kinetic_energy = validate_quantities(func=kinetic_energy_lite, mass=u.kg, velocity=u.m/u.s, validations_on_return=u.Joule)

kinetic_energy(mass=1233 * u.kg, velocity=39.34 * u.m/u.s)

Much better, but we still have a hidden bug.  What happens is the mass is negative?

In [ ]:
kinetic_energy(mass=-1233 * u.kg, velocity=39.34 * u.m/u.s)

Well, that does not make much sense, but `validate_quantites` can still help us out...

In [ ]:
kinetic_energy = validate_quantities(
    func=kinetic_energy_lite, 
    mass={"units": u.kg, "can_be_negative": False},
    velocity=u.m/u.s, 
    validations_on_return=u.Joule,
)

kinetic_energy(mass=-1233 * u.kg, velocity=39.34 * u.m/u.s)

Well, that makes a lot more physical sense!

We can still make this a lot cleaner by using Python's sugar syntax for decorators.

In [ ]:
@validate_quantities(
    mass={"units": u.kg, "can_be_negative": False},
    velocity=u.m/u.s, 
    validations_on_return=u.Joule,
)
def kinetic_energy(mass, velocity):
    return 0.5 * mass * velocity**2

# even cleaner

@validate_quantities(mass={"can_be_negative": False})
def kinetic_energy(mass: u.kg, velocity: u.m/u.s) -> u.Joule:
    return 0.5 * mass * velocity**2

Using non-standard units

In [ ]:
mass = 2718 * u.imperial.lb  # mass of a Delorean
velocity = 88 * u.imperial.mile / u.hour

kinetic_energy(mass=mass, velocity=velocity)

## `@particle_input`

Well, that was cool and all but not really usefull if we are trying to find the kinetic energy of a particle.  `plasmapy` does make this a little easier with the `Particle` class.

In [ ]:
from plasmapy.particles import Particle

par = Particle("Fe")
par.mass

In [ ]:
kinetic_energy(mass=par.mass, velocity=velocity)

But we can do better by using the `@particle_input` decorator. 

In [ ]:
from plasmapy.particles import particle_input

@validate_quantities
@particle_input
def kinetic_energy(velocity: u.m/u.s, particle: Particle) -> u.Joule:
    return 0.5 * particle.mass * velocity**2

In [ ]:
kinetic_energy(velocity=velocity, particle="Fe")


## `plasmapy.formulary` Unit Agnoticism

`astropy.unts` (along with some magic) allowed us to develop a formulary that is agnostic to input units, and will always return values in SI units.

In [ ]:
import astropy.units as u

from plasmapy.formulary import Debye_length

Let's start with SI units

In [ ]:
T_e = 115000 * u.K  # about 10 eV
n_e = 2e18 * u.m**-3

Debye_length(T_e=T_e, n_e=n_e)

Let's do the same calculation, but with non-SI units

In [ ]:
T_e_in_eV = T_e.to(u.eV, equivalencies=u.temperature_energy())
n_e_in_lsec3 = n_e.to(u.lsec**-3)

(T_e_in_eV, n_e_in_lsec3)

In [ ]:
Debye_length(T_e=T_e_in_eV, n_e=n_e_in_lsec3)

## `@validate_quantites` - The Units Enforcer

The reasone we have unit agnosticism is because we have develped an units enforce.  This enforcer, `@validate_quantites`, knows how to recognize input unit types, interpret what the function is expecting, and do the proper conversion or enforcement.

Let's start with a function that calculates the kinetic energy of a mass.

In [ ]:
def kinetic_energy_lite(mass, velocity):
    return 0.5 * mass * velocity**2

kinetic_energy_lite(mass=1233, velocity=39.34)

But we have not unit enforement...

In [ ]:
from plasmapy.utils.decorators import validate_quantities

In [ ]:
kinetic_energy = validate_quantities(func=kinetic_energy_lite, mass=u.kg, velocity=u.m/u.s)

kinetic_energy(mass=1233, velocity=39.34)

Now we are getting warnings that input arguments are not given with units, and the output units are a total mess.

In [ ]:
kinetic_energy(mass=1233 * u.kg, velocity=39.34 * u.m/u.s)

The output units are still a mess, so we can do better...

In [ ]:
kinetic_energy = validate_quantities(func=kinetic_energy_lite, mass=u.kg, velocity=u.m/u.s, validations_on_return=u.Joule)

kinetic_energy(mass=1233 * u.kg, velocity=39.34 * u.m/u.s)

Much better, but we still have a hidden bug.  What happens is the mass is negative?

In [ ]:
kinetic_energy(mass=-1233 * u.kg, velocity=39.34 * u.m/u.s)

Well, that does not make much sense, but `validate_quantites` can still help us out...

In [ ]:
kinetic_energy = validate_quantities(
    func=kinetic_energy_lite, 
    mass={"units": u.kg, "can_be_negative": False},
    velocity=u.m/u.s, 
    validations_on_return=u.Joule,
)

kinetic_energy(mass=-1233 * u.kg, velocity=39.34 * u.m/u.s)

Well, that makes a lot more physical sense!

We can still make this a lot cleaner by using Python's sugar syntax for decorators.

In [ ]:
@validate_quantities(
    mass={"units": u.kg, "can_be_negative": False},
    velocity=u.m/u.s, 
    validations_on_return=u.Joule,
)
def kinetic_energy(mass, velocity):
    return 0.5 * mass * velocity**2

# even cleaner

@validate_quantities(mass={"can_be_negative": False})
def kinetic_energy(mass: u.kg, velocity: u.m/u.s) -> u.Joule:
    return 0.5 * mass * velocity**2

Using non-standard units

In [ ]:
mass = 2718 * u.imperial.lb  # mass of a Delorean
velocity = 88 * u.imperial.mile / u.hour

kinetic_energy(mass=mass, velocity=velocity)

## `@particle_input`

Well, that was cool and all but not really usefull if we are trying to find the kinetic energy of a particle.  `plasmapy` does make this a little easier with the `Particle` class.

In [ ]:
from plasmapy.particles import Particle

par = Particle("Fe")
par.mass

In [ ]:
kinetic_energy(mass=par.mass, velocity=velocity)

But we can do better by using the `@particle_input` decorator. 

In [ ]:
from plasmapy.particles import particle_input

@validate_quantities
@particle_input
def kinetic_energy(velocity: u.m/u.s, particle: Particle) -> u.Joule:
    return 0.5 * particle.mass * velocity**2

In [ ]:
kinetic_energy(velocity=velocity, particle="Fe")